최신 체인 구성방법 V0.3
```
LLMChain, SequentailChain 등과 같이 클래스 의존도를 줄임
Runnable 공통인터페이스를 통해 일관성을 유지
핵심 : Runnable + Composition --> 프롬프트 | 모델 | 파서
```

<!-- %pip install langchain openai python-dotenv -->

In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
os.getenv('OPENAI_API_KEY')[:10]  # Just to ensure the key is loaded


'sk-proj-2d'

단일체인 : prompt -> llm -> 출력파서(상품설명)

In [12]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 1. 프롬프트 탬플릿을 정의
product_prompt = PromptTemplate.from_template(
    "제품 이름 : {product_name}\n"
    "이 제품의 특징과 장점을 매력적인 한 단락으로 설명해 주세요"
)
# 2. 출력파서 정의
product_output_parser = StrOutputParser()
# 3. llm 정의
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
    )

# 4. LCEL 체인 구성 프롬프트 -> 모델 -> 출력파서
product_chain = product_prompt | llm | product_output_parser
result = product_chain.invoke({"product_name": "커피"})
print(result)

커피는 아침의 시작을 활기차게 열어주는 마법의 음료입니다. 깊고 풍부한 향은 감각을 깨우고, 첫 모금에서 느껴지는 부드러운 쓴맛과 은은한 단맛의 조화는 하루의 피로를 잊게 해줍니다. 카페인은 집중력과 에너지를 높여주어 업무 효율을 극대화하며, 다양한 원두와 추출 방식에 따라 각기 다른 맛과 향을 즐길 수 있어 개인의 취향에 맞춘 커피 경험을 제공합니다. 또한, 커피는 사회적 연결의 매개체로, 친구나 동료와의 대화 속에서 따뜻한 유대감을 형성하는 데 기여합니다. 이러한 커피의 매력은 단순한 음료를 넘어 일상의 소소한 행복을 선사합니다.


다중체인 : 체인 합성 및 Runnable 병합 (이메일 생성)
```
둘 이상의 llm 호출을 연결해서 복잡한 작업을 수행
"주어진 상황에 대한 이메일 작성" --> 제목
제목을 활용해서 이메일 본문을 작성 --> 본문
chain composition
```

In [13]:
# 프롬프트 1 : 사용자로부터 받은 이메일 요청내용을 입력받아서 "이메일 제목"을 한문장으로 생성하는 명령
# llm 호출 -> 이메일 제목출력('프로젝트 진행상황 회의 일정 안내')
# 중간 출력 변환 : 생성된 제목 문자열을 {subject} 키를 갖는 dictionary 변환
# 프롬프트 2 : 이메일 제목을 입력받아서 "이메일 본문"을 생성하는 명령
# llm 호출 -> 이메일 본문 출력('안녕하세요, 프로젝트 진행상황 회의 일정을 안내드립니다. 회의는 다음 주 월요일 오전 10시에 진행될 예정입니다. 자세한 사항은 첨부된 문서를 참고해 주세요.')

In [ ]:
from langchain_core.runnables import RunnablePassthrough
# 1. 이메일 생성용 프롬프트 정의
subject_prompt = PromptTemplate.from_template(
    '다음 요청 내용을 바탕으로 이메일 제목을 만들어주세요.\n'
    '{content}'
)
# 2. 이메일 제목 출력 파서 정의
body_prompt = PromptTemplate.from_template(
    '위에서 생성된 제목을 활용해서 팀에게 보내는 정중한 이메일 본문을 작성해 주세요.\n'
    '제목 : {subject}\n'
    '본문 :'

)
# 3. 두 프롬프트를 결합한 체인 구성
email_chain = (
    subject_prompt
    | llm
    | {"subject": RunnablePassthrough()} # 출력된 제목을 subject키로 매핑
    | body_prompt
    | llm
    | StrOutputParser()
)
# 4. 이메일 요청 내용 입력
result = email_chain.invoke({
    "content": "프로젝트 진행상황 회의 일정 안내"
})
print(result)

안녕하세요 팀 여러분,

프로젝트 진행상황을 점검하고 향후 계획을 논의하기 위해 회의를 개최하고자 합니다. 아래의 일정을 확인하시고 참석 부탁드립니다.

- 일시: [날짜 및 시간]
- 장소: [회의 장소]
- 주요 안건: 프로젝트 진행상황 점검, 문제점 논의 및 해결 방안, 향후 일정 계획

회의 준비를 위해 각자 맡은 부분의 진행 상황을 정리해 주시면 감사하겠습니다. 추가로 논의가 필요한 사항이 있다면 미리 공유해 주세요.

바쁘시겠지만 꼭 참석해 주시길 부탁드립니다. 감사합니다.

좋은 하루 보내세요.

[보내는 사람 이름] 드림


조건분기 : 입력조건에 따라 요약 또는 이메일 작성

In [24]:
from langchain_core.runnables import RunnableBranch

summary_prompt = PromptTemplate.from_template(
    "다음 내용을 요약해 주세요.\n"
    "{content}"
)
summary_chain = summary_prompt | llm | StrOutputParser()
# 2. email chain은 재화용
# 3. 분기조건 함수 정의 -> Runnable로 래핑
def is_summary_request(user_input: str) -> bool:
    return user_input.strip().startswith("요약:")
condition = RunnablePassthrough(is_summary_request)
# 분기 체인
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
branch_chain = RunnableBranch(
    (condition, summary_chain), email_chain
)
# 다양한 조건
input1 = "요약: 오늘 회의에서는 다양한 주제에 대한 토론이 있었는데, 특히 일정 변경과 예산 관련 내용이 다수였습니다."
input2 = "이메일 : 다음주 월요일 프로젝트 회의 일정을 팀에게 공지해줘"

result1 = branch_chain.invoke(input1)
result2 = branch_chain.invoke(input2)
print(f"result1 : {result1}")
print(f"result2 : {result2}")

result1 : 오늘 회의에서는 여러 주제가 논의되었으며, 특히 일정 변경과 예산에 관한 논의가 주를 이루었습니다.
result2 : 다음 주 월요일에 있을 프로젝트 회의 일정을 팀에게 공지해 달라는 요청입니다.
